## Automatically import data
### this is taking in "Master_email_list(manually_maintained)"
### this is managing a file that I have manually put together and maintain for this particular project. 

In [ ]:
## If using Google colab 
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [ ]:
import pandas as pd
import io
import numpy as np
# test

In [ ]:
# Set up read path, read file, write path, write file
curr_project_name = "project_erudition_emails_2"
notebook_path = "/mnt/c/Users/isaac/Desktop/mturks_assistant_github/mturks_assistant/notebooks"

input_filename1 = 'main_batch_blast2_directory-to-dental_clubs_Completed File.xlsx'
input_filename2 = 'main_batch_blast2_directory-to-dental_clubs_Revison_Completed File.xlsx'
read_path1 = notebook_path+"/../projects/"+curr_project_name+"/5_hits__clubs/"+input_filename1
read_path2 = notebook_path+"/../projects/"+curr_project_name+"/5_hits__clubs/"+input_filename2

output_filename = 'dropped_duplicates_and_empties.xlsx'
write_path = notebook_path+"/../projects/"+curr_project_name+"/6_hits__clubs_labeled/"+output_filename

In [ ]:
df1 = pd.read_excel(read_path1,sheet_name="Cameron complete data")
df1 = df1.drop_duplicates()
df1['version'] = 'original'

df2 = pd.read_excel(read_path2,sheet_name="Cameron complete data")
df2 = df2.drop_duplicates()
df2['version'] = 'revised'

print(df1.shape)
print(df2.shape)


In [ ]:
df1_yes = df1.loc[df1['isaac_decision']=='yes']
df2_yes = df2.loc[df2['isaac_decision']=='yes']
print(df1_yes.shape)
print(df2_yes.shape)

In [ ]:
df_yes_full = pd.concat([df1_yes,df2_yes])
df_yes_full = df_yes_full.drop_duplicates(['university',	'city', 'country',	'directory_link',	'club_name',	'isaac_decision',	'Unnamed: 6',	'club_link',	'comments, etc.'])
print(df_yes_full.shape)

In [ ]:
# export the dataframe
df_yes_full.to_excel(write_path)

## This here is to resolve any redundancies between a "pre-health" list and a "pre-dental" list, allocating those to the "predental" group

In [ ]:
print(df2.shape)
count = 0
for name,group in df2.groupby(['club','winner_email']):
  if df2.loc[ (df2['club']==name[0]) & (df2['winner_email'] == name[1]) ].shape[0] >1:
    count += 1

    if df2.loc[ (df2['club']==name[0]) & (df2['winner_email'] == name[1]) ].shape[0] == 2:
      
      predental = []
      prehealth = []
      for index, row in df2.loc[ (df2['club']==name[0]) & (df2['winner_email'] == name[1]) ].iterrows():
        
        if (df2.loc[index,'predental_or_prehealth']=='predental'):
          predental.append(index)
        else:
          prehealth.append(index)
      
      
      if len(predental) == 1:
        df2.drop(prehealth[0], inplace=True)
      elif len(prehealth) == 2:
        df2.drop(prehealth[0], inplace=True)
      else:
        df2.drop(predental[0], inplace=True)

    else:
      print("redundancy not due to prehealth and predental captures (this should not be the case)")
      print("name",name)
      print("group",group)
    
print(df2.shape)

print(count)

In [ ]:
# Write file
df2.to_excel(write_path)